In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.tree import export_text
import mglearn
from dashboard_one import *
from dash_model_two import *
from feature_selection import *
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

### sleep_____VS_____final diagnosischanges of each patient in each phase


#### sleep_brain_finaldxch.csv
#### 

In [2]:
com_col = ['Phase', 'RID', 'VISCODE','PTID','RID_Phase']
target = 'final_dxch'

In [3]:
sleep_brain_finaldxch = pd.read_csv('sleep_brain_finaldxch.csv').iloc[:,1:].drop(['NPIKSEV'],axis=1)
sleep_brain_finaldxch = sleep_brain_finaldxch[sleep_brain_finaldxch['final_dxch'].notna()].reset_index().drop(['index'],axis=1)   # keep the rows where DXCHANGE is not nan
sleep_brain_finaldxch

RID  Phase VISCODE        PTID   RID_Phase  NPIK1  NPIK2  NPIK3  \
0         2  ADNI2     v06  011_S_0002     2_ADNI2    NaN    NaN    NaN   
1         2  ADNI2     v11  011_S_0002     2_ADNI2    NaN    NaN    NaN   
2         2  ADNI2     v21  011_S_0002     2_ADNI2    NaN    NaN    NaN   
3         2  ADNI2     v41  011_S_0002     2_ADNI2    NaN    NaN    NaN   
4         2  ADNI2     v51  011_S_0002     2_ADNI2    NaN    NaN    NaN   
...     ...    ...     ...         ...         ...    ...    ...    ...   
14740  6978  ADNI3      y1  021_S_6978  6978_ADNI3    NaN    NaN    NaN   
14741  6999  ADNI3     m12  035_S_6999  6999_ADNI3    NaN    NaN    NaN   
14742  6999  ADNI3      y1  035_S_6999  6999_ADNI3    NaN    NaN    NaN   
14743  7000  ADNI3     m12  035_S_7000  7000_ADNI3    NaN    NaN    NaN   
14744  7000  ADNI3      y1  035_S_7000  7000_ADNI3    NaN    NaN    NaN   

       NPIK4  NPIK5  ...  ratio_PTAU_bl  Ventricles_reduction_per_year  \
0        NaN    NaN  ...            NaN                            NaN   
1        NaN    NaN  ...            NaN                            NaN   
2        NaN    NaN  ...            NaN                            NaN   
3        NaN    NaN  ...            NaN                            NaN   
4        NaN    NaN  ...            NaN                            NaN   
...      ...    ...  ...            ...                            ...   
14740    NaN    NaN  ...            NaN                            NaN   
14741    NaN    NaN  ...            NaN                            NaN   
14742    NaN    NaN  ...            NaN                            NaN   
14743    NaN    NaN  ...            NaN                            NaN   
14744    NaN    NaN  ...            NaN                            NaN   

       Hippocampus_reduction_per_year  wholebrain_reduction_per_year  \
0                                 NaN                            NaN   
1                                 NaN                            NaN   
2                                 NaN                            NaN   
3                                 NaN                            NaN   
4                                 NaN                            NaN   
...                               ...                            ...   
14740                             NaN                            NaN   
14741                             NaN                            NaN   
14742                             NaN                            NaN   
14743                             NaN                            NaN   
14744                             NaN                            NaN   

       Entorhinal_reduction_per_year  Fusiform_reduction_per_year  \
0                                NaN                          NaN   
1                                NaN                          NaN   
2                                NaN                          NaN   
3                                NaN                          NaN   
4                                NaN                          NaN   
...                              ...                          ...   
14740                            NaN                          NaN   
14741                            NaN                          NaN   
14742                            NaN                          NaN   
14743                            NaN                          NaN   
14744                            NaN                          NaN   

       ICV_reduction_per_year  ABETA_reduction_per_year  \
0                         NaN                       NaN   
1                         NaN                       NaN   
2                         NaN                       NaN   
3                         NaN                       NaN   
4                         NaN                       NaN   
...                       ...                       ...   
14740                     NaN                       NaN   
14741                     NaN                       

In [4]:
sleep_brain_finaldxch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14745 entries, 0 to 14744
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   RID                             14745 non-null  int64  
 1   Phase                           14745 non-null  object 
 2   VISCODE                         13649 non-null  object 
 3   PTID                            14745 non-null  object 
 4   RID_Phase                       14745 non-null  object 
 5   NPIK1                           884 non-null    float64
 6   NPIK2                           883 non-null    float64
 7   NPIK3                           882 non-null    float64
 8   NPIK4                           882 non-null    float64
 9   NPIK5                           882 non-null    float64
 10  NPIK6                           883 non-null    float64
 11  NPIK7                           883 non-null    float64
 12  NPIK8                           

In [5]:
sleep_brain_finaldxch.columns

Index(['RID', 'Phase', 'VISCODE', 'PTID', 'RID_Phase', 'NPIK1', 'NPIK2',
       'NPIK3', 'NPIK4', 'NPIK5', 'NPIK6', 'NPIK7', 'NPIK8', 'NPIK9A',
       'NPIK9B', 'NPIK9C', 'NPIKTOT', 'insomnia', 'OSA', 'final_dxch',
       'DXCHANGE', 'ratio_Ventricles_bl', 'ratio_Hippocampus_bl',
       'ratio_WholeBrain_bl', 'ratio_Entorhinal_bl', 'ratio_Fusiform_bl',
       'ratio_ICV_bl', 'ratio_ABETA_bl', 'ratio_TAU_bl', 'ratio_PTAU_bl',
       'Ventricles_reduction_per_year', 'Hippocampus_reduction_per_year',
       'wholebrain_reduction_per_year', 'Entorhinal_reduction_per_year',
       'Fusiform_reduction_per_year', 'ICV_reduction_per_year',
       'ABETA_reduction_per_year', 'TAU_reduction_per_year',
       'PTAU_reduction_per_year'],
      dtype='object')

### brain_biomarker______VS______final_dxch

In [6]:
col_lst = [ 'final_dxch','NPIK1', 'NPIK2',
       'NPIK3', 'NPIK4', 'NPIK5', 'NPIK6', 'NPIK7', 'NPIK8', 'NPIK9A',
       'NPIK9B', 'NPIK9C', 'NPIKTOT', 'insomnia', 'OSA']
bio_lst = [ 'final_dxch','ratio_ABETA_bl', 'ratio_TAU_bl','ratio_PTAU_bl']
sleep_dxch = sleep_brain_finaldxch[com_col + col_lst].set_index(com_col).dropna(how='any',axis=0).reset_index()
#biomarkers to dxch
bio_dxch = sleep_brain_finaldxch[com_col + bio_lst].set_index(com_col).dropna(how='any',axis=0).reset_index()
df = sleep_dxch.copy()

In [7]:
np.sum(df.isna())   # check nan

Phase         0
RID           0
VISCODE       0
PTID          0
RID_Phase     0
final_dxch    0
NPIK1         0
NPIK2         0
NPIK3         0
NPIK4         0
NPIK5         0
NPIK6         0
NPIK7         0
NPIK8         0
NPIK9A        0
NPIK9B        0
NPIK9C        0
NPIKTOT       0
insomnia      0
OSA           0
dtype: int64

In [8]:
df.groupby(target).count()

Phase  RID  VISCODE  PTID  RID_Phase  NPIK1  NPIK2  NPIK3  NPIK4  \
final_dxch                                                                     
AD-AD         185  185      185   185        185    185    185    185    185   
AD-MCI          6    6        6     6          6      6      6      6      6   
CN-AD           2    2        2     2          2      2      2      2      2   
CN-CN         186  186      186   186        186    186    186    186    186   
CN-MCI         31   31       31    31         31     31     31     31     31   
MCI-AD         96   96       96    96         96     96     96     96     96   
MCI-CN         32   32       32    32         32     32     32     32     32   
MCI-MCI       339  339      339   339        339    339    339    339    339   

            NPIK5  NPIK6  NPIK7  NPIK8  NPIK9A  NPIK9B  NPIK9C  NPIKTOT  \
final_dxch                                                                
AD-AD         185    185    185    185     185     185     185      185   
AD-MCI          6      6      6      6       6       6       6        6   
CN-AD           2      2      2      2       2       2       2        2   
CN-CN         186    186    186    186     186     186     186      186   
CN-MCI         31     31     31     31      31      31      31       31   
MCI-AD         96     96     96     96      96      96      96       96   
MCI-CN         32     32     32     32      32      32      32       32   
MCI-MCI       339    339    339    339     339     339     339      339   

            insomnia  OSA  
final_dxch                 
AD-AD            185  185  
AD-MCI             6    6  
CN-AD              2    2  
CN-CN            186  186  
CN-MCI            31   31  
MCI-AD            96   96  
MCI-CN            32   32  
MCI-MCI          339  339

- unselect the labels with too little data: AD-MCI, CN-AD, CN-MCI, MCI-CN
- the possible groups: ['CN-CN','MCI-AD', 'MCI-MCI','AD-AD']

In [9]:
df_2g = df.loc[df[target].isin(['MCI-AD', 'MCI-CN'])].reset_index().drop(['index'],axis=1)
df_2g.shape

(128, 20)

In [10]:
df_3g = df.loc[df[target].isin(['MCI-AD', 'MCI-CN','MCI-MCI'])].reset_index().drop(['index'],axis=1)
df_3g.shape

(467, 20)

### undersampling and modeling

- functions
    - models(df,drop_lst,target) : under sampling, split, scale, pca, models
    - cv_models(df,drop_lst,target,k): under sampling, NOT SPLIT, scale, pca, models with cross-validation 

In [11]:
drop_lst = ['Phase', 'RID', 'VISCODE', 'PTID','RID_Phase',target]
k=10

# - 'MCI-AD': 32, 'MCI-CN': 32

- 'MCI-AD': 32, 'MCI-CN': 32}
- #original dataset: random forest 55trees. average weighted f1-score of 10-cross validation:0.645

In [12]:
models(df_2g,drop_lst,target)

After undersampling data size is 64 ; Resampled dataset shape Counter({'MCI-AD': 32, 'MCI-CN': 32})

8 principle components are needed to explain 90% of the data

Output dataframes sequence: X_train,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_train,y_test
- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, Training set f1-score:0.668, Test set f1-score: 0.544
          - saga_L1, Training set f1-score:0.675, Test set f1-score: 0.632
          - newton-cg_L2, Training set f1-score:0.668, Test set f1-score: 0.544
       - C = 0.01
          - lbfgs_L2, Training set f1-score:0.775, Test set f1-score: 0.538
          - saga_L1, Training set f1-score:0.658, Test set f1-score: 0.700
          - newton-cg_L2, Training set f1-score:0.775, Test set f1-score: 0.538
       - C = 0.1
          - lbfgs_L2, Training set f1-score:0.826, Test set f1-score: 0.620
          - saga_L1, Training set f1-score:0.678, Test set f1-score: 0.544
        

          - 15trees. f1-score on training data: 0.941 f1-score on test data: 0.692
          - 20trees. f1-score on training data: 0.941 f1-score on test data: 0.704
          - 25trees. f1-score on training data: 0.941 f1-score on test data: 0.772
          - 30trees. f1-score on training data: 0.941 f1-score on test data: 0.646
          - 35trees. f1-score on training data: 0.941 f1-score on test data: 0.772
          - 40trees. f1-score on training data: 0.941 f1-score on test data: 0.704
          - 45trees. f1-score on training data: 0.941 f1-score on test data: 0.704
          - 50trees. f1-score on training data: 0.941 f1-score on test data: 0.772
          - 55trees. f1-score on training data: 0.941 f1-score on test data: 0.704
          - 60trees. f1-score on training data: 0.941 f1-score on test data: 0.772
          - 65trees. f1-score on training data: 0.941 f1-score on test data: 0.772
          - 70trees. f1-score on training data: 0.941 f1-score on test data: 0.620
    

In [13]:
cv_models(df_2g,drop_lst,target,k)

After undersampling data size is 64 ; Resampled dataset shape Counter({'MCI-AD': 32, 'MCI-CN': 32})

8 principle components are needed to explain 90% of the data

- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.575
          - saga_L1, average weighted f1-score of 10-cross validation:0.335
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.575
       - C = 0.01
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.679
          - saga_L1, average weighted f1-score of 10-cross validation:0.335
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.679
       - C = 0.1
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.657
          - saga_L1, average weighted f1-score of 10-cross validation:0.550
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.657
       - C = 1
          - lb

          - 25trees. average weighted f1-score of 10-cross validation:0.608
          - 30trees. average weighted f1-score of 10-cross validation:0.573
          - 35trees. average weighted f1-score of 10-cross validation:0.560
          - 40trees. average weighted f1-score of 10-cross validation:0.545
          - 45trees. average weighted f1-score of 10-cross validation:0.579
          - 50trees. average weighted f1-score of 10-cross validation:0.594
          - 55trees. average weighted f1-score of 10-cross validation:0.645
          - 60trees. average weighted f1-score of 10-cross validation:0.645
          - 65trees. average weighted f1-score of 10-cross validation:0.630
          - 70trees. average weighted f1-score of 10-cross validation:0.645
          - 75trees. average weighted f1-score of 10-cross validation:0.615
          - 80trees. average weighted f1-score of 10-cross validation:0.615
          - 85trees. average weighted f1-score of 10-cross validation:0.595
          - 

In [14]:
res = usampling_scale_data(df_2g,drop_lst,target)     
X = res[0]
y = res[3]
clf = RandomForestClassifier(n_estimators =55, random_state = 5862)
title_label = '10-fold crossvalidation of random forest with (55 trees)'
feature_importance(X,y,clf,10,title_label)

After undersampling data size is 64 ; Resampled dataset shape Counter({'MCI-AD': 32, 'MCI-CN': 32})

8 principle components are needed to explain 90% of the data

Features sorted by their score for each estimator 


avg_importance  importance_1  importance_2  importance_3  \
NPIK9C          0.169285      0.175372      0.176845      0.178429   
NPIKTOT         0.164782      0.138031      0.169764      0.166604   
NPIK9A          0.125314      0.117180      0.113772      0.144927   
NPIK7           0.097935      0.099356      0.081024      0.091218   
NPIK4           0.073968      0.079721      0.072308      0.077496   
NPIK2           0.070770      0.067885      0.057345      0.064606   
OSA             0.065909      0.062323      0.080417      0.054939   
NPIK1           0.063909      0.068700      0.067979      0.063788   
NPIK9B          0.056478      0.054347      0.054403      0.066054   
NPIK6           0.051542      0.060569      0.048002      0.044408   
insomnia        0.026551      0.031282      0.030695      0.016003   
NPIK8           0.021849      0.033265      0.027018      0.018820   
NPIK5           0.011707      0.011967      0.020429      0.012707   
NPIK3           0.000000      0.000000      0.000000      0.000000   

          importance_4  importance_5  importance_6  importance_7  \
NPIK9C        0.188315      0.152009      0.162420      0.172689   
NPIKTOT       0.181928      0.164134      0.174648      0.140933   
NPIK9A        0.116643      0.108415      0.150398      0.125842   
NPIK7         0.108890      0.106521      0.088514      0.120594   
NPIK4         0.034353      0.106709      0.051238      0.091312   
NPIK2         0.059795      0.077553      0.085834      0.058295   
OSA           0.076364      0.063528      0.069786      0.071539   
NPIK1         0.074105      0.062933      0.052686      0.065329   
NPIK9B        0.061393      0.060827      0.050738      0.045611   
NPIK6         0.047796      0.057962      0.056895      0.043298   
insomnia      0.027672      0.013631      0.034530      0.023982   
NPIK8         0.009456      0.014098      0.022313      0.026023   
NPIK5         0.013288      0.011681      0.000000      0.014553   
NPIK3         0.000000      0.000000      0.000000      0.000000   

          importance_8  importance_9  importance_10  
NPIK9C        0.156833      0.168890       0.161042  
NPIKTOT       0.175871      0.166405       0.169502  
NPIK9A        0.146206      0.118190       0.111563  
NPIK7         0.086539      0.100130       0.096567  
NPIK4         0.082136      0.067289       0.077113  
NPIK2         0.078725      0.083895       0.073771  
OSA           0.045402      0.057305       0.077493  
NPIK1         0.054913      0.067617       0.061044  
NPIK9B        0.054602      0.054097       0.062707  
NPIK6         0.056001      0.047143       0.053345  
insomnia      0.031404      0.029131       0.027181  
NPIK8         0.017463      0.028164       0.021876  
NPIK5         0.013907      0.011744       0.006795  
NPIK3         0.000000      0.000000       0.000000

MCI-AD': 32, 'MCI-CN': 32, 'MCI-MCI': 32

In [15]:
models(df_3g,drop_lst,target)

After undersampling data size is 96 ; Resampled dataset shape Counter({'MCI-AD': 32, 'MCI-CN': 32, 'MCI-MCI': 32})

9 principle components are needed to explain 90% of the data

Output dataframes sequence: X_train,X_test,X_train_scaled,X_test_scaled,X_train_pca,X_test_pca,y_train,y_test
- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, Training set f1-score:0.575, Test set f1-score: 0.478
          - saga_L1, Training set f1-score:0.524, Test set f1-score: 0.400
          - newton-cg_L2, Training set f1-score:0.575, Test set f1-score: 0.538
       - C = 0.01
          - lbfgs_L2, Training set f1-score:0.562, Test set f1-score: 0.601
          - saga_L1, Training set f1-score:0.524, Test set f1-score: 0.400
          - newton-cg_L2, Training set f1-score:0.562, Test set f1-score: 0.601
       - C = 0.1
          - lbfgs_L2, Training set f1-score:0.584, Test set f1-score: 0.601
          - saga_L1, Training set f1-score:0.522, Test set f1-score:

          - 30trees. f1-score on training data: 0.894 f1-score on test data: 0.489
          - 35trees. f1-score on training data: 0.894 f1-score on test data: 0.543
          - 40trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 45trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 50trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 55trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 60trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 65trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 70trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 75trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 80trees. f1-score on training data: 0.894 f1-score on test data: 0.495
          - 85trees. f1-score on training data: 0.894 f1-score on test data: 0.495
    

In [16]:
cv_models(df_3g,drop_lst,target,k)

After undersampling data size is 96 ; Resampled dataset shape Counter({'MCI-AD': 32, 'MCI-CN': 32, 'MCI-MCI': 32})

9 principle components are needed to explain 90% of the data

- Using original dataset:
    - Logistic regression
       - C = 0.001
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.300
          - saga_L1, average weighted f1-score of 10-cross validation:0.159
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.300
       - C = 0.01
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.356
          - saga_L1, average weighted f1-score of 10-cross validation:0.159
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.356
       - C = 0.1
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.327
          - saga_L1, average weighted f1-score of 10-cross validation:0.274
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.327
       - C = 1

C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.335
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.325
       - C = 100
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.335


C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.359
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.335
       - C = 1000
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.335


...

          - saga_L1, average weighted f1-score of 10-cross validation:0.344
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.335
    - Decision tree
          - tree depth: 1.000. average weighted f1-score of 10-cross validation:0.233
          - tree depth: 2.000. average weighted f1-score of 10-cross validation:0.316
          - tree depth: 3.000. average weighted f1-score of 10-cross validation:0.373
          - tree depth: 4.000. average weighted f1-score of 10-cross validation:0.373
          - tree depth: 5.000. average weighted f1-score of 10-cross validation:0.422
          - tree depth: 6.000. average weighted f1-score of 10-cross validation:0.377
          - tree depth: 7.000. average weighted f1-score of 10-cross validation:0.361
          - tree depth: 8.000. average weighted f1-score of 10-cross validation:0.344
          - tree depth: 9.000. average weighted f1-score of 10-cross validation:0.371
          - tree depth: 10.000. average weighted 

C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.313
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.312
       - C = 1000
          - lbfgs_L2, average weighted f1-score of 10-cross validation:0.312


C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\weipi\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


          - saga_L1, average weighted f1-score of 10-cross validation:0.305
          - newton-cg_L2, average weighted f1-score of 10-cross validation:0.312
    - Decision tree
          - tree depth: 1.000. average weighted f1-score of 10-cross validation:0.381
          - tree depth: 2.000. average weighted f1-score of 10-cross validation:0.384
          - tree depth: 3.000. average weighted f1-score of 10-cross validation:0.357
          - tree depth: 4.000. average weighted f1-score of 10-cross validation:0.343
          - tree depth: 5.000. average weighted f1-score of 10-cross validation:0.369
          - tree depth: 6.000. average weighted f1-score of 10-cross validation:0.332
          - tree depth: 7.000. average weighted f1-score of 10-cross validation:0.374
          - tree depth: 8.000. average weighted f1-score of 10-cross validation:0.365
          - tree depth: 9.000. average weighted f1-score of 10-cross validation:0.333
          - tree depth: 10.000. average weighted 